# Main class - `EEGSignalGenerator.m`

In [ ]:
%%file EEGSignalGenerator.m
classdef EEGSignalGenerator
    % A class which can be used for generating signals from activity sources.
    %
    % EEGSignalGenerator Properties:
    %    SETUP - setup for simulation,
    %    MODEL - model variables,
    %    MATS  - input data files.
    %
    % EEGSignalGenerator Methods:
    %    init - initilize neccesary toolboxes,
    %    setparameters - setting up parameters for simulation,
    %    setsignals - creating signals.
    
    properties(GetAccess='public', SetAccess='public')
        % Properties of the class
        
        SETUP;
        MODEL;
        MATS;
    end

    methods
        function obj = EEGSignalGenerator()
            % Constructor of the EEGSignalGenerator class

            obj.SETUP = [];
            obj.MODEL = [];
        end
        function SETUP = get.SETUP(obj)
            % Getter for variable SETUP

            SETUP = obj.SETUP;
        end
        function obj = set.SETUP(obj, NEWSETUP)
            % Setter for variable SETUP

            obj.SETUP = NEWSETUP;
        end
        function r = init(obj)
            % Initialize toolboxes.

            MATS = inittoolboxes();
            r = obj;
            r.MATS = MATS;
        end
        function obj = setparameters(obj, SETUP)
            % Sets parameters for simulation.

            obj.SETUP = SETUP;
        end
        function obj = setsignals(obj)
            % Create signals. Feel free to change.
            
            disp('CYBERCRAFT:: Generation of timeseries for bioelectrical activity of interest');
            obj.MODEL = generatetimeseriessourceactivity(obj.SETUP, obj.MODEL);

            disp('CYBERCRAFT:: Generation of timeseries for bioelectrical interference noise');
            obj.MODEL = generatetimeseriesinterferencenoise(obj.SETUP, obj.MODEL);

            disp('CYBERCRAFT:: Generation of timeseries for bioelectrical background noise');
            obj.MODEL = generatetimeseriesbackgroundactivity(obj.SETUP, obj.MODEL);

            disp('CYBERCRAFT:: Measurement noise generation');
            obj.MODEL = generatetimeseriesmeasurementnoise(obj.SETUP, obj.MODEL, obj.MATS);
        end
    end
end

# Auxiliary functions

In [ ]:
%%file inittoolboxes.m
function MATS = inittoolboxes()
    % Auxiliary toolboxes added to path.
    % Required *.mat files containing meshes are loaded here.

    if exist('~/toolboxes/', 'dir') == 7
        TMP_TOOLB_PATH = '~/toolboxes/';
    else
        warningMessage = sprintf('CYBERCRAFT:: Warning:: toolboxes directory was not found (use ''~/toolboxes/'')');
    end;
    if exist('TMP_TOOLB_PATH', 'var')
        disp(['CYBERCRAFT:: looking for toolboxes in: ', TMP_TOOLB_PATH])
    end;
     addpath([TMP_TOOLB_PATH, '/arfit']);
     addpath([TMP_TOOLB_PATH, '/mvarica']);
     
         % Loading meshes
    
    load('./mat/sel_msh.mat');                     % head compartments geometry (cortex)
    load('./mat/sel_geo_deep_thalami.mat');        % mesh containing candidates for lacation of deep sources (based on thalami)
    load('./mat/sel_geo_deep_icosahedron642.mat'); % mesh containing candidates for lacation of deep sources (based on icosahedron642)
    load('./mat/sel_atl.mat');                     % cortex geometry with (anatomical) ROI parcellation
    load('./mat/sel_vol.mat');                     % volume conduction model (head-model)
    load('./mat/sel_ele.mat');                     % geometry of electrode positions
    load('./mat/sel_src.mat');                     % all cx leadfields

    MATS = [];
    MATS.sel_msh = sel_msh;
    MATS.sel_geo_deep_thalami = sel_geo_deep_thalami;
    MATS.sel_geo_deep_icosahedron642 = sel_geo_deep_icosahedron642;
    MATS.sel_atl = sel_atl; % Atlas from Brainstorm, about 15000 vertices
    MATS.sel_vol = sel_vol; % Volumne conduction model
    MATS.sel_ele = sel_ele; % Electrodes in FieldTrip model
    MATS.sel_src = sel_src; % Sources leadfields
end

In [ ]:
%%file makeSimSig.m
function sim_sig = makeSimSig(SETUP, set_COL, varargin)
    % MVAR-based signal generations; the signal is then halved into 
    % 'pre' and 'pst' parts

    sim_sig.inf        = '';
    sim_sig.S00        = [];
    sim_sig.P00        = [];
    sim_sig.M00        = [];
    sim_sig.A00        = zeros([0, 1]);
    sim_sig.mdlStabH   = [];
    sim_sig.mdlStabM   = [];
    sim_sig.w00        = [];
    sim_sig.C00        = [];
    sim_sig.n00        = [];
    sim_sig.K00        = [];
    sim_sig.sigSRC_pre = permute([], [3, 2, 1]);
    sim_sig.sigSRC_pst = permute([], [3, 2, 1]);
    sim_sig.w01        = [];
    sim_sig.A01        = zeros([0, 1]);
    sim_sig.C01        = [];
    sim_sig.SBC01      = [];
    sim_sig.FPE01      = [];
    sim_sig.th01       = [];
    sim_sig.srate      = SETUP.SRATE;
    if ~isempty(varargin)
        sim_sig.inf = varargin{end};
    end
    sim_sig.S00 = sum(SETUP.SRCS(:, set_COL)) + SETUP.DEEP(1, set_COL); % nbr of signals
    if sim_sig.S00 > 0
        sim_sig.P00 = SETUP.P00;  % order of the MVAR model used to generate time-courses
        if SETUP.SEED
            rng(SETUP.SEEDS(4));
        end
        sim_sig.M00 = EEGStaticMethods().diagonMask(sim_sig.S00, SETUP.FRAC); % MVAR coefficients mask
        if SETUP.SEED
            rng(SETUP.SEEDS(5));
        end
        sim_sig.A00 = EEGStaticMethods().stableMVAR(sim_sig.S00, sim_sig.P00, sim_sig.M00, SETUP.RNG, SETUP.STAB, SETUP.ITER, SETUP.DEBUG);
        [sim_sig.mdlStabH, sim_sig.mdlStabM] = EEGStaticMethods().rawIsStableMVAR(sim_sig.A00, 1);
        sim_sig.w00 = zeros(sim_sig.S00, 1); % expected value for time-courses
        sim_sig.C00 = eye(sim_sig.S00);     % covariance of MVAR model's "driving noise"
        sim_sig.n00 = SETUP.n00;                     % number of time samples
        sim_sig.K00 = SETUP.K00;                     % number of independent realizations of the driving AR models
        if SETUP.SEED
            rng(SETUP.SEEDS(3));
        end
        sim_sig.sigSRC_pre = 10 * arsim(sim_sig.w00, sim_sig.A00, sim_sig.C00, [sim_sig.n00, 2*sim_sig.K00], 1e3); % 10e-6; % Current density (10nA*mm)
        sim_sig.sigSRC_pst = sim_sig.sigSRC_pre(:, :, [end/2 + 1:end]);
        sim_sig.sigSRC_pre = sim_sig.sigSRC_pre(:, :, [1:end/2]);
        if SETUP.TELL
            disp('Fitting MVAR model to generated signal...');
        end;
        [sim_sig.w01, sim_sig.A01, sim_sig.C01, sim_sig.SBC01, sim_sig.FPE01, sim_sig.th01] = arfit(sim_sig.sigSRC_pst, sim_sig.P00 - round(sim_sig.P00/2), sim_sig.P00 + round(sim_sig.P00/2));
        [sim_sig.w01, sim_sig.A01, sim_sig.C01, sim_sig.SBC01, sim_sig.FPE01, sim_sig.th01] = arfit(sim_sig.sigSRC_pst, sim_sig.P00, sim_sig.P00);
    end
end

In [ ]:
%%file generatetimeseriessourceactivity.m
function model = generatetimeseriessourceactivity(SETUP, MODEL)
    % Generates signal of interest and, if required,
    % adds ERP deterministic signal to the signal of interest.

    model = MODEL;
    model.sim_sig_SrcActiv = makeSimSig(SETUP, 1, 'MVAR based signal for sources activity');

    if SETUP.ERPs > 0
        model.sim_sig_SrcActiv.sigSRC_pst_orig = model.sim_sig_SrcActiv.sigSRC_pst;
        model.sim_sig_SrcActiv.sigSRC_pre_orig = model.sim_sig_SrcActiv.sigSRC_pre;

        tmp_LB = -5;
        tmp_UB = 7;
        tmp_NN = SETUP.n00;
        tmp_PP = 1;
        for ee = 1:1:SETUP.ERPs
            tmp_PP = mod(ee, 3) + 1;
            model.sim_sig_SrcActiv.ERPs(ee, :) = gauswavf(tmp_LB, tmp_UB, tmp_NN, tmp_PP);
        end
        model.sim_sig_SrcActiv.ERPs_pst = transpose(model.sim_sig_SrcActiv.ERPs);
        model.sim_sig_SrcActiv.ERPs_pst(size(model.sim_sig_SrcActiv.sigSRC_pst, 1), size(model.sim_sig_SrcActiv.sigSRC_pst, 2)) = 0;
        model.sim_sig_SrcActiv.ERPs_pst = cat(3, repmat(model.sim_sig_SrcActiv.ERPs_pst, [1, 1, SETUP.K00]));

        model.sim_sig_SrcActiv.sigSRC_pst = model.sim_sig_SrcActiv.sigSRC_pst + 20 * model.sim_sig_SrcActiv.ERPs_pst;
    end
end

In [ ]:
%%file generatetimeseriesinterferencenoise.m
function model = generatetimeseriesinterferencenoise(SETUP, MODEL)
    % Generates time series interference noise.

    model = MODEL;
    model.sim_sig_IntActiv = model.sim_sig_SrcActiv;

    model.sim_sig_IntActiv.inf = 'MVAR based signal for interference (biological) noise activity sources';
    tmp_IntActivCol = 2;

    if SETUP.TELL
        disp('Getting base for IntActiv...');
    end;
    model.sim_sig_IntActiv.sigSRC_pre = -model.sim_sig_IntActiv.sigSRC_pre;
    model.sim_sig_IntActiv.sigSRC_pst = -model.sim_sig_IntActiv.sigSRC_pst;
    [model.sim_sig_IntActiv.w01, model.sim_sig_IntActiv.A01, model.sim_sig_IntActiv.C01, model.sim_sig_IntActiv.SBC01, model.sim_sig_IntActiv.FPE01, model.sim_sig_IntActiv.th01] = deal([]);

    if SETUP.TELL
        disp('Populating signals for IntActiv...');
    end;
    model.sim_sig_IntActiv.sigSRC_pre = repmat(model.sim_sig_IntActiv.sigSRC_pre, 1, ceil( (sum(SETUP.SRCS(:, tmp_IntActivCol)) + SETUP.DEEP(1, tmp_IntActivCol))/size(model.sim_sig_IntActiv.sigSRC_pre, 2)), 1);
    model.sim_sig_IntActiv.sigSRC_pst = repmat(model.sim_sig_IntActiv.sigSRC_pst, 1, ceil( (sum(SETUP.SRCS(:, tmp_IntActivCol)) + SETUP.DEEP(1, tmp_IntActivCol))/size(model.sim_sig_IntActiv.sigSRC_pst, 2)), 1);

    if SETUP.TELL
        disp('Reducing dimensionality of IntActiv...');
    end;
    model.sim_sig_IntActiv.sigSRC_pre = model.sim_sig_IntActiv.sigSRC_pre(:, 1:(sum(SETUP.SRCS(:, tmp_IntActivCol)) + SETUP.DEEP(1, tmp_IntActivCol)), :);
    model.sim_sig_IntActiv.sigSRC_pst = model.sim_sig_IntActiv.sigSRC_pst(:, 1:(sum(SETUP.SRCS(:, tmp_IntActivCol)) + SETUP.DEEP(1, tmp_IntActivCol)), :);

    if SETUP.TELL
        disp('Backing-up IntActiv before white noise admixture...');
    end;
    model.sim_sig_IntActiv.sigSRC_pre_b4admix = model.sim_sig_IntActiv.sigSRC_pre;
    model.sim_sig_IntActiv.sigSRC_pst_b4admix = model.sim_sig_IntActiv.sigSRC_pst;

    if SETUP.WhtNoiseAddFlg
        if SETUP.TELL
            disp('Adding some white noise to the biological noise activity...');
        end;
        if SETUP.SEED
            rng(SETUP.SEEDS(1));
        end
        model.sim_sig_IntActiv.noiseAdmix_pre = randn(size(model.sim_sig_IntActiv.sigSRC_pre));
        if SETUP.SEED
            rng(SETUP.SEEDS(1));
        end
        model.sim_sig_IntActiv.noiseAdmix_pst = randn(size(model.sim_sig_IntActiv.sigSRC_pst));
        for kk = 1:SETUP.K00
            model.sim_sig_IntActiv.noiseAdmix_pre(:, :, kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_IntActiv.sigSRC_pre(:, :, kk), model.sim_sig_IntActiv.noiseAdmix_pre(:, :, kk), SETUP.WhtNoiseAddSNR);
            model.sim_sig_IntActiv.noiseAdmix_pst(:, :, kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_IntActiv.sigSRC_pst(:, :, kk), model.sim_sig_IntActiv.noiseAdmix_pst(:, :, kk), SETUP.WhtNoiseAddSNR);
        end
        model.sim_sig_IntActiv.sigSRC_pre = model.sim_sig_IntActiv.sigSRC_pre + model.sim_sig_IntActiv.noiseAdmix_pre;
        model.sim_sig_IntActiv.sigSRC_pst = model.sim_sig_IntActiv.sigSRC_pst + model.sim_sig_IntActiv.noiseAdmix_pst;
    end

    if SETUP.TELL
        disp('Fitting MVAR model to generated signal...');
    end;
    [model.sim_sig_IntActiv.w01, model.sim_sig_IntActiv.A01, model.sim_sig_IntActiv.C01, model.sim_sig_IntActiv.SBC01, model.sim_sig_IntActiv.FPE01, model.sim_sig_IntActiv.th01] = arfit(model.sim_sig_IntActiv.sigSRC_pst, model.sim_sig_IntActiv.P00 - round(model.sim_sig_IntActiv.P00/2), model.sim_sig_IntActiv.P00 + round(model.sim_sig_IntActiv.P00/2));
    model.sim_sig_IntActiv.w01 = model.sim_sig_IntActiv.w01;
    model.sim_sig_IntActiv.A01 = model.sim_sig_IntActiv.A01;
    model.sim_sig_IntActiv.C01 = model.sim_sig_IntActiv.C01;
    model.sim_sig_IntActiv.SBC01 = model.sim_sig_IntActiv.SBC01;
    model.sim_sig_IntActiv.FPE01 = model.sim_sig_IntActiv.FPE01;
    model.sim_sig_IntActiv.th01 = model.sim_sig_IntActiv.th01;

    if SETUP.DEBUG
       model.sim_sig_IntActiv = rmfield(model.sim_sig_IntActiv, {'sigSRC_pre_b4admix', 'sigSRC_pst_b4admix', 'noiseAdmix_pre', 'noiseAdmix_pst'});
    end
end

In [ ]:
%%file generatetimeseriesbackgroundactivity.m
function model = generatetimeseriesbackgroundactivity(SETUP, MODEL)
    % Generates time series background activity.

    model = MODEL;
    model.sim_sig_BcgActiv = makeSimSig(SETUP, 3, 'MVAR based signal for background (biological) noise activity sources');
end

In [ ]:
%%file generatetimeseriesmeasurementnoise.m
function model = generatetimeseriesmeasurementnoise(SETUP, MODEL, MATS)
    % Generates time series measurement noise.

    model = MODEL;
    model.sim_sig_MesNoise.inf = 'RANDN based signal for measurement noise';
    if SETUP.SEED
        rng(SETUP.SEEDS(2));
    end
    model.sim_sig_MesNoise.sigSNS_pre = randn([SETUP.n00, size(MATS.sel_ele.chanpos, 1), SETUP.K00]);
    if SETUP.SEED
        rng(SETUP.SEEDS(2));
    end
    model.sim_sig_MesNoise.sigSNS_pst = randn([SETUP.n00, size(MATS.sel_ele.chanpos, 1), SETUP.K00]);
end